In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Rotation, Horizontall Flip, Crop

In [ ]:
!pip install pandas openpyxl


In [ ]:
import cv2
import os
import random
import pandas as pd
import numpy as np

def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    return cv2.warpAffine(img, M, (w, h))

def horizontal_flip(img):
    return cv2.flip(img, 1)  # 1 berarti flip horizontal

def resize_to_target(img, target_size=(1024, 1024)):
    h, w = img.shape[:2]

    padding = 30

    # Hitung rasio aspek
    rasio_w = (target_size[0]-padding) / w
    rasio_h = (target_size[1]-padding) / h

    # Tentukan rasio terkecil untuk memastikan gambar tidak terpotong
    rasio = min(rasio_w, rasio_h)

    new_w = int(w * rasio)
    new_h = int(h * rasio)

    resized_img = cv2.resize(img, (new_w, new_h))

    # Jika gambar lebih kecil dari 512x512 setelah resize, tambahkan padding
    delta_w = target_size[0] - new_w
    delta_h = target_size[1] - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]
    return cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)


# Baca metadata dari file Excel
metadata_path = '/content/drive/MyDrive/metadata.xlsx'
metadata_path_new = '/content/drive/MyDrive/metadata_new.xlsx'
metadata_df = pd.read_excel(metadata_path)


# Lokasi direktori gambar
dir_path = '/content/drive/MyDrive/vektor'
output_path = '/content/drive/MyDrive/vektor/processed'
if not os.path.exists(output_path):
    os.makedirs(output_path)

metadata_df['processed_filename'] = ""

file_name = []
judul = []
text = []

num = 1

for i in range(10):
  for _, row in metadata_df.iterrows():
      image_file = row['file_name']
      img_path = os.path.join(dir_path, image_file)
      if os.path.exists(img_path):
          img = cv2.imread(img_path)

          # Binary image
          img = cv2.threshold(img, 0, 255, cv2.THRESH_TOZERO)[1]

          # Invers warna gambar
          img = 255 - img

          # Crop ke 512x512
          img = resize_to_target(img)

          # Rotasi gambar
          if np.random.randn() > 0.3:
            img = rotation(img, 8)

          # Flip horizontal
          if np.random.randn() > 0.6:
            img = horizontal_flip(img)

          # Invers gambar lagi sebelum disimpan
          img_final_inverse = 255 - img

          # Simpan gambar yang diproses
          processed_filename = "processed_" + row['judul'] + str(num) + ".jpg"
          save_path = os.path.join(output_path, processed_filename)
          cv2.imwrite(save_path, img_final_inverse)

          # Tetapkan nama file yang telah diproses ke kolom baru di metadata_df
          # row['processed_filename'] = processed_filename
          file_name += [processed_filename]
          judul += [row['judul']]
          text += [row['text']]

          # Menampilkan informasi pemrosesan
          print(f"Processed {image_file} and saved as {processed_filename} in {output_path}")
      else:
          print(f"{image_file} not found in {dir_path}.")

      num += 1

new_df = pd.DataFrame({
    'file_name' : file_name,
    'judul' : judul,
    'text' : text
})

# metadata_df = metadata_df.append(new_df)

print("Processing completed for all images.")

# Simpan metadata_df kembali ke metadata.xlsx
new_df.to_excel(metadata_path_new, index=False)

Processed Adenium.jpg and saved as processed_adenium1.jpg in /content/drive/MyDrive/vektor/processed
Processed Boket Ayam Alas.jpg and saved as processed_anggrek2.jpg in /content/drive/MyDrive/vektor/processed
Processed Anggrek Hutan.jpg and saved as processed_anggrek bulan3.jpg in /content/drive/MyDrive/vektor/processed
Processed anggrek pohon.jpg and saved as processed_anggrek pohon4.jpg in /content/drive/MyDrive/vektor/processed
Processed Anggrek Rusa.jpg and saved as processed_anggrek rusa5.jpg in /content/drive/MyDrive/vektor/processed
Processed Begisar.jpg and saved as processed_ayam bekisar6.jpg in /content/drive/MyDrive/vektor/processed
Processed pring sedapur.jpg and saved as processed_bambu7.jpg in /content/drive/MyDrive/vektor/processed
Processed bintang laut.jpg and saved as processed_bintang laut8.jpg in /content/drive/MyDrive/vektor/processed
Processed IKAN BADUT.jpg and saved as processed_biota laut9.jpg in /content/drive/MyDrive/vektor/processed
Processed Burung.jpg and

In [ ]:
# img_inverse = 255 - img